In [1]:
# テンプレートの内容を利用し，国旗画像のURLを取得せよ．
# （ヒント: MediaWiki APIのimageinfoを呼び出して，ファイル参照をURLに変換すればよい）

In [2]:
import utils
import re
import json
import requests

In [3]:
content = utils.extract_UK()[0]

In [4]:
# 基礎情報テンプレートの抽出
pattern1 = re.compile(r'''
    ^\{\{基礎情報.*?$
    (.*?)
    ^\}\}$
''', re.MULTILINE + re.VERBOSE + re.DOTALL)
result1 = pattern1.findall(content)

In [5]:
# 抽出したテンプレートからkeyとvalueを抽出
pattern2 = re.compile(r'''
    ^\|
    (.+?)
    \s*=\s*
    (.+?)
    (?: # キャプチャ対象外
        (?=\n\|) | (?=\n$) # 肯定の先読み  
    )
''', re.MULTILINE + re.VERBOSE + re.DOTALL)
result2 = pattern2.findall(result1[0])

In [6]:
## マークアップの削除
def remove_markup(target):
    ## 強調マークアップ
    patternIntense = re.compile(r'''
        (\'{2,5})
        (.*?)
        (\1)
    ''', re.MULTILINE + re.VERBOSE)
    ## 内部リンク、ファイル
    patternInternalLink = re.compile(r'''
        \[\[
        (.*?)
        \]\]
    ''', re.MULTILINE + re.VERBOSE)
    ## Tempalte:Lang {{lang|言語タグ|文字列}}
    patternLang = re.compile(r'''
        \{\{lang
        \|
        (?:
           [^|]*? # '|'以外の文字が0文字以上 []は集合,^で補集合を表す。
        )
        \|
        (.*?)
        \}\}
    ''', re.MULTILINE + re.VERBOSE)
    ## <br>, <ref>
    patternHTML = re.compile(r'''
        <
        \/? # 終了タグかどうか
        [br|ref]
        [^>]*?
        >
    ''', re.MULTILINE + re.VERBOSE)
    ## 外部リンク
    patternExternalLink = re.compile(r'''
        \[http:\/\/ # 'http://'
        (?:
            [^\s]*?
            \s
        )?
        ([^\]]*?)
        \]
    ''', re.MULTILINE + re.VERBOSE)
    
    target = patternIntense.sub(r'\2', target)
    target = patternInternalLink.sub(r'\1', target)
    target = patternLang.sub(r'\1', target)
    target = patternHTML.sub('', target)
    target = patternExternalLink.sub(r'\1', target)
    return target

In [7]:
result3 = {}
for title, text in result2:
    result3[title] = remove_markup(text)

In [8]:
# 国旗画像のvalueを取得
fname_flag = result3['国旗画像']

In [9]:
S = requests.Session()
URL = 'https://www.mediawiki.org/w/api.php?'
PARAMS = {
    'action': 'query',
    'format': 'json',
    'prop': 'imageinfo',
    'titles': 'File:' + fname_flag,
    'iiprop': 'url'
}
R = S.get(url=URL, params=PARAMS)
DATA = R.json()

url = DATA['query']['pages'].popitem()[1]['imageinfo'][0]['url']
print(url)

https://upload.wikimedia.org/wikipedia/commons/a/ae/Flag_of_the_United_Kingdom.svg
